# Utilización de varias técnicas para realizar sentiment analysis:
- RNN
- Word embedding
- Transfer learning

Autores:
- Lawrence Javier Minguillán Van Kapel
- Alberto Sánchez Bonastre

## Cargamos los datos de google collab

In [20]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Almacenamiento de nuestro dataset en la variable data

In [21]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Comillas-ICAI/data.csv")
data.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


# RNN for sentiment analysis

En este modelo de análisis de sentimientos, hemos seguido los siguientes pasos:

1. Preparación de los datos: Comenzamos importando las bibliotecas necesarias, incluyendo pandas y numpy para el manejo de datos, así como las clases relacionadas con la construcción y entrenamiento de modelos de redes neuronales recurrentes (RNN) utilizando TensorFlow.
2. Carga y limpieza de datos: Importamos nuestros datos desde un archivo CSV y los cargamos en un DataFrame. Luego, eliminamos cualquier fila que tenga valores faltantes utilizando el método dropna.
3. Codificación de etiquetas de sentimiento: Utilizamos LabelEncoder para convertir las etiquetas de sentimiento de texto en valores numéricos. Esto es necesario para que el modelo pueda trabajar con ellas durante el entrenamiento.
4. División de datos: Dividimos nuestros datos en conjuntos de entrenamiento y prueba utilizando train_test_split de scikit-learn. Esto nos permite evaluar la capacidad de generalización del modelo.
5. Tokenización y secuenciación de texto: Utilizamos Tokenizer y pad_sequences de TensorFlow para convertir las secuencias de texto en secuencias de números enteros, preparándolas para ser procesadas por la red neuronal.
6. Construcción del modelo de red neuronal recurrente (RNN): Creamos un modelo secuencial utilizando Sequential() de TensorFlow. Este modelo consta de una capa de embedding, una capa LSTM y una capa densa. La capa de embedding convierte los números enteros en vectores de números reales, la capa LSTM procesa secuencias de entrada y la capa densa realiza la clasificación final.
7. Compilación del modelo: Compilamos el modelo especificando el optimizador, la función de pérdida y las métricas que se utilizarán durante el entrenamiento. En este caso, utilizamos el optimizador Adam y la función de pérdida de entropía cruzada binaria.
7. Entrenamiento del modelo: Entrenamos el modelo utilizando el conjunto de entrenamiento, con validación cruzada para monitorear el rendimiento. Utilizamos EarlyStopping para detener el entrenamiento si la pérdida en el conjunto de validación deja de disminuir, lo que ayuda a evitar el sobreajuste.
8. Evaluación del modelo: Finalmente, evaluamos el rendimiento del modelo utilizando el conjunto de prueba y mostramos la precisión obtenida. Esto nos da una idea de qué tan bien el modelo puede predecir los sentimientos en datos que no ha visto durante el entrenamiento.

In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Leer el archivo CSV
df = data
df.dropna(inplace=True)

# Sentence,Sentiment
# Codificar las etiquetas de sentimiento
label_encoder = LabelEncoder()
df['Sentiment'] = label_encoder.fit_transform(df['Sentiment'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['Sentence'], df['Sentiment'], test_size=0.2, random_state=42)

# Tokenización y secuenciación de texto
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100  # Longitud máxima de la secuencia
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Crear el modelo de red neuronal recurrente
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train_pad, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Precisión del modelo:", accuracy)

Epoch 1/20
117/117 [==============================] - 59s 432ms/step - loss: -1.6232 - accuracy: 0.5452 - val_loss: -2.9213 - val_accuracy: 0.5016
Epoch 2/20
117/117 [==============================] - 48s 406ms/step - loss: -3.1987 - accuracy: 0.5455 - val_loss: -4.1350 - val_accuracy: 0.5016
Epoch 3/20
117/117 [==============================] - 41s 353ms/step - loss: -4.2576 - accuracy: 0.5455 - val_loss: -5.2781 - val_accuracy: 0.5016
Epoch 4/20
117/117 [==============================] - 42s 361ms/step - loss: -5.2997 - accuracy: 0.5455 - val_loss: -6.4481 - val_accuracy: 0.5016
Epoch 5/20
117/117 [==============================] - 41s 350ms/step - loss: -6.3755 - accuracy: 0.5455 - val_loss: -7.6390 - val_accuracy: 0.5016
Epoch 6/20
117/117 [==============================] - 41s 347ms/step - loss: -7.4359 - accuracy: 0.5455 - val_loss: -8.7988 - val_accuracy: 0.5016
Epoch 7/20
117/117 [==============================] - 40s 346ms/step - loss: -8.5199 - accuracy: 0.5455 - val_loss: -1

*   Probamos el modelo con 10 epocs -> produce 34% de precisión - 10 minutos
*   Probamos el modelo con 20 epocs -> produce 52% de precisión - 20 minutos
*   Probamos el modelo con 40 epocs -> produce 54% de precisión - 40 minutos

Tambien se ha cambiado el tamaño de los batches del entreanmiento reduciendolos a 4, 6 y 8 aumnetando las epocs a 50 pero el resultado final deja una precisión resultante de 54% y un tiempo de procesamiento de aproximadamente 1 hora. Por lo que en comparativa con los modelos determinamos que el "codo" de tiempo de procesamiento vs accuracy recae en el modelo entrenado con 20 epocas y un batch size de 32.

In [59]:
# Guardamos el modelo
model.save('./model_rnn.keras')

Como la precisión de nuestro modelo actual es baja, alrededor del 54%, una estrategia que podemos utilizar para mejorarla es incorporar word embeddings pre-entrenados. Estos embeddings son representaciones vectoriales de palabras que capturan relaciones semánticas y contextuales entre palabras en un espacio vectorial. Utilizar word embeddings pre-entrenados nos permite aprovechar el conocimiento lingüístico aprendido de grandes corpus de texto y aplicarlo a nuestra tarea específica de análisis de sentimientos.

# Word embeddings pre-entrenados - GloVe embeddings en un modelo LSTM

Ahora realizaremos un análisis de similitud de coseno utilizando vectores de palabras pre-entrenados, específicamente el conjunto de vectores GloVe. Los pasos que se han seguido son los siguientes:

1. Carga de vectores de palabras pre-entrenados: Se carga un archivo de vectores de palabras pre-entrenados (en este caso, se utiliza el archivo 'glove.42B.300d.txt') que contiene las representaciones vectoriales de palabras en un espacio de alta dimensionalidad.
2. Creación de un índice de embeddings: Se crea un diccionario que mapea cada palabra a su vector de embedding correspondiente. Esto permite acceder rápidamente a los vectores de palabras durante el procesamiento.

In [101]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

embedding_file = '/content/drive/MyDrive/Comillas-ICAI/glove.42B.300d.txt'

embedding_index = {}
with open(embedding_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Función para obtener el vector promedio de una frase
def get_sentence_vector(sentence):
    words = sentence.split()
    sentence_vector = np.zeros((len(words), 300))  # Tamaño del vector de palabra GloVe
    for i, word in enumerate(words):
        if word in embedding_index:
            sentence_vector[i] = embedding_index[word]
    return np.mean(sentence_vector, axis=0)

3. Definición de una función para obtener el vector de una frase: Se define una función llamada get_sentence_vector(sentence) que toma una frase como entrada y devuelve su vector de embedding promedio. Para ello, se divide la frase en palabras y se busca cada palabra en el índice de embeddings para obtener su vector correspondiente. Se calcula el vector promedio de todas las palabras en la frase.

In [106]:
import pickle

with open('./embedding_index.pickle', 'wb') as handle:
    pickle.dump(embedding_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

4. Guardado del índice de embeddings en un archivo pickle: El índice de embeddings se guarda en un archivo pickle llamado 'embedding_index.pickle' para su posterior uso.
5. División de datos en conjuntos de entrenamiento y prueba: Se dividen los datos de entrada en conjuntos de entrenamiento y prueba utilizando la función train_test_split de scikit-learn.
6. Obtención de vectores de características para frases de entrenamiento y prueba: Se utiliza la función get_sentence_vector para obtener los vectores de características correspondientes a las frases de entrenamiento y prueba.
7. Entrenamiento de un clasificador (por ejemplo, SVM): Se entrena un clasificador, en este caso un Support Vector Classifier (SVC) con un kernel lineal, utilizando los vectores de características de las frases de entrenamiento y sus respectivas etiquetas de sentimiento.
8. Evaluación del clasificador en el conjunto de prueba: Se evalúa el rendimiento del clasificador en el conjunto de prueba calculando su precisión, que es la proporción de predicciones correctas sobre el total de predicciones.

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X = data['Sentence']
y = data['Sentiment']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtener vectores de características para frases de entrenamiento y prueba
X_train_vectors = np.array([get_sentence_vector(sentence) for sentence in X_train])
X_test_vectors = np.array([get_sentence_vector(sentence) for sentence in X_test])

# Entrenar un clasificador (por ejemplo, SVM)
clf = SVC(kernel='linear')
clf.fit(X_train_vectors, y_train)

# Evaluar el clasificador en el conjunto de prueba
accuracy = clf.score(X_test_vectors, y_test)
print("Precisión del clasificador:", accuracy)


Precisión del clasificador: 0.7125748502994012


Mediante el uso de word embeddings pre-entrenados, hemos logrado mejorar significativamente la precisión de nuestro modelo de análisis de sentimientos. Mientras que nuestro modelo anterior, basado en redes neuronales recurrentes (RNN), alcanzó una precisión del 54%, la incorporación de word embeddings nos ha permitido elevar esta métrica a un impresionante 71%.

Mostramos un ejemplo del vector embedding de la palabra 'lawrence'.

In [103]:
word = 'lawrence'
embedding_vector = embedding_index.get(word)
print(f'Embedding vector for "{word}": {embedding_vector}')

Embedding vector for "lawrence": [-2.9427e-01 -5.7970e-02 -1.5971e-02  3.9954e-01 -6.2847e-01 -4.1986e-02
 -6.4807e-01 -5.7400e-01  5.4430e-02  1.4884e-01  1.3946e-01  2.3571e-01
  1.7424e-01 -9.1451e-02 -6.7695e-01 -2.6709e-01  2.8099e-01  9.1666e-02
  6.0602e-02 -4.2523e-01 -3.0175e-01 -3.4897e-02 -2.9880e-01 -3.2512e-01
 -1.2465e-01  2.9856e-01 -1.4872e-01 -1.9790e-01 -2.2669e-01  3.3482e-01
  4.0580e-01  5.5613e-02  1.5809e-01 -1.0308e-01  1.0452e-01 -1.3513e-01
  1.5482e-01  4.3143e-01 -9.6263e-02 -3.6093e-01 -1.4344e-01  1.5833e-01
  5.6850e-01 -3.8225e-01  2.2130e-01  4.2912e-02 -5.2333e-01 -1.3488e-02
  5.6025e-01  2.6849e-01  2.8533e-01 -2.7855e-01 -2.9604e-02 -1.1529e-01
  1.0453e-01  3.0762e-01  2.5936e-01  1.8043e-01 -1.6598e-01  3.1576e-01
 -2.3928e-01  1.7130e-01  5.7337e-01 -2.9780e-01 -2.1183e-01 -3.1928e-01
  1.3632e-01 -7.3382e-02  3.3463e-01 -6.0898e-01  2.7280e-01 -4.5832e-01
  1.8715e-01 -4.0205e-02  1.5931e-01  2.5965e-01 -3.7205e-01 -4.2004e-01
 -1.4238e-01  6.58

In [104]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predicciones del conjunto de prueba
y_pred = clf.predict(X_test_vectors)

# Calcular precisión
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

# Calcular precisión
precision = precision_score(y_test, y_pred, average='weighted')
print("Precisión (weighted):", precision)

# Calcular exhaustividad
recall = recall_score(y_test, y_pred, average='weighted')
print("Exhaustividad (weighted):", recall)

# Calcular F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score (weighted):", f1)

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(conf_matrix)


Precisión: 0.7125748502994012
Precisión (weighted): 0.6905424965525682
Exhaustividad (weighted): 0.7125748502994012
F1-score (weighted): 0.6924136374063784
Matriz de confusión:
[[ 40  90  45]
 [ 24 550  48]
 [ 26 103 243]]


Los resultados obtenidos muestran un rendimiento moderado del modelo de clasificación, con una precisión del 71.26% y métricas relacionadas que indican una capacidad de predicción aceptable pero no excepcional. Si bien el modelo ha logrado identificar correctamente la mayoría de las muestras positivas en el conjunto de prueba, hay margen de mejora evidente. En este sentido, se plantea explorar una estrategia de transfer learning para potenciar el rendimiento del modelo. El transfer learning implica aprovechar el conocimiento aprendido por un modelo previamente entrenado en una tarea relacionada y aplicarlo a una nueva tarea.

# Transfern learning - BERT

El modelo de transfer learning implementado sigue los siguientes pasos:

1. Carga de Datos: Los datos de entrada, que consisten en frases (sentences) y etiquetas de sentimiento (labels), son cargados desde un dataframe o una estructura similar.
2. División de Datos: Los datos se dividen en conjuntos de entrenamiento y prueba utilizando la función train_test_split de scikit-learn. Esto asegura que el modelo pueda ser entrenado en una parte de los datos y evaluado en otra parte separada.
3. Tokenización: Se utiliza el tokenizador de BERT (Bidirectional Encoder Representations from Transformers) para convertir las frases de entrada en tokens y generar los correspondientes encodings necesarios para el modelo.
4. Creación del Dataset: Se define una clase SentimentDataset que hereda de Dataset de PyTorch. Esta clase se encarga de almacenar los encodings de las frases y las etiquetas de sentimiento y de proporcionar estos datos en el formato adecuado para el entrenamiento y la evaluación del modelo.
5. Carga del Modelo Pre-entrenado: Se carga un modelo pre-entrenado de BERT para clasificación de secuencias (BertForSequenceClassification) utilizando la función from_pretrained de la librería transformers de Hugging Face. Se especifica el número de etiquetas de clasificación, que en este caso son 3 (positivo, negativo, neutral).
6. Entrenamiento del Modelo: Se utiliza un DataLoader para cargar los datos de entrenamiento en lotes y se itera sobre estos lotes para realizar el entrenamiento del modelo. Durante el entrenamiento, se calcula la pérdida (loss) y se realizan las actualizaciones de los pesos del modelo utilizando el optimizador AdamW.
7. Evaluación del Modelo: Similar al entrenamiento, se utiliza un DataLoader para cargar los datos de prueba en lotes y se itera sobre estos lotes para realizar la evaluación del modelo. Se calculan las predicciones del modelo y se comparan con las etiquetas verdaderas para calcular la precisión del modelo utilizando la métrica de precisión (accuracy).

In [49]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Cargar datos
data = data
sentences = data['Sentence'].tolist()
labels = data['Sentiment'].tolist()

# Dividir datos en conjunto de entrenamiento y prueba
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)

# Convertir etiquetas a números enteros
label_map = {"positive": 0, "negative": 1, "neutral": 2}
train_labels = [label_map[label] for label in train_labels]
test_labels = [label_map[label] for label in test_labels]


# Tokenizar los datos
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_sentences, truncation=True, padding=True)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# Cargar modelo pre-entrenado
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3 clases de sentimiento: positivo, negativo, neutral

# Entrenamiento
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.train()
for batch in train_loader:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluación
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
all_preds = []
all_labels = []
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].tolist()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).tolist()
    all_preds.extend(preds)
    all_labels.extend(labels)

accuracy = accuracy_score(all_labels, all_preds)
print("Accuracy:", accuracy)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.7921300256629598


Con una precisión del 79%, el modelo de transfer learning basado en BERT ha superado significativamente tanto al modelo que utiliza embeddings pre-entrenados como al modelo basado en RNN.

Comparado con el modelo de embeddings, que alcanzó una precisión del 71%, el modelo de transfer learning basado en BERT ha logrado un aumento notable del 8% en la precisión. Esto indica una mejora significativa en la capacidad del modelo para discernir y clasificar correctamente las expresiones de sentimiento en el texto en comparación con el enfoque anterior de embeddings pre-entrenados.

Además, el rendimiento del modelo BERT es mucho mejor que el de la RNN, que solo logró una precisión del 54%. Esta diferencia de 25 puntos porcentuales resalta claramente la superioridad del enfoque de transfer learning basado en BERT sobre el enfoque anterior de RNN en términos de precisión en la tarea de análisis de sentimientos.

In [50]:
# Guardar el modelo entrenado
model_path = "./model_transferlearning"

# Guardar el modelo y el tokenizador
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print("Modelo entrenado y tokenizador guardados en:", model_path)


Modelo entrenado y tokenizador guardados en: ./model_transferlearning


In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

# Calcular métricas
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds)


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Matriz de Confusión:")
print(conf_matrix)

Accuracy: 0.7921300256629598
Precision: 0.8020280890262151
Recall: 0.7921300256629598
F1-score: 0.7934267354349172
Matriz de Confusión:
[[337   8  27]
 [ 39 103  33]
 [ 60  76 486]]


Los resultados del modelo de transfer learning son muy positivos en general. Con una precisión del 79.21%, el modelo clasifica correctamente cerca del 79% de las muestras en el conjunto de prueba. La precisión del 80.20% indica su capacidad para identificar con precisión las muestras positivas. Además, el recall del 79.21% muestra su habilidad para recuperar la mayoría de las muestras positivas. El F1-score del 79.34% refleja un buen equilibrio entre precisión y recall. La matriz de confusión revela una clasificación correcta para la mayoría de las muestras, con pocos errores. En resumen, estos resultados sugieren que el modelo de transfer learning es efectivo y confiable para la tarea de análisis de sentimientos.